In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import plotly
import plotly.express as px
import pandas as pd
import numpy as np
import os
import re

In [9]:
%cd /content/drive/MyDrive/Colab Notebooks/data-analysis

/content/drive/MyDrive/Colab Notebooks/data-analysis


## Dataset preprocessing

### Kaggle

In [10]:
kaggle_path = get_csv_from_gdrive(KAGGLE_URL)
kaggle_multiple_choice = pd.read_csv("data/multiple_choice_responses.csv", low_memory=False)
kaggle_multiple_choice.head()

,Time from Start to Finish (seconds),Q1,Q2,Q2_OTHER_TEXT,Q3,Q4,Q5,Q5_OTHER_TEXT,Q6,Q7,Q8,Q9_Part_1,Q9_Part_2,Q9_Part_3,Q9_Part_4,Q9_Part_5,Q9_Part_6,Q9_Part_7,Q9_Part_8,Q9_OTHER_TEXT,Q10,Q11,Q12_Part_1,Q12_Part_2,Q12_Part_3,Q12_Part_4,Q12_Part_5,Q12_Part_6,Q12_Part_7,Q12_Part_8,Q12_Part_9,Q12_Part_10,Q12_Part_11,Q12_Part_12,Q12_OTHER_TEXT,Q13_Part_1,Q13_Part_2,Q13_Part_3,Q13_Part_4,Q13_Part_5,...,Q31_OTHER_TEXT,Q32_Part_1,Q32_Part_2,Q32_Part_3,Q32_Part_4,Q32_Part_5,Q32_Part_6,Q32_Part_7,Q32_Part_8,Q32_Part_9,Q32_Part_10,Q32_Part_11,Q32_Part_12,Q32_OTHER_TEXT,Q33_Part_1,Q33_Part_2,Q33_Part_3,Q33_Part_4,Q33_Part_5,Q33_Part_6,Q33_Part_7,Q33_Part_8,Q33_Part_9,Q33_Part_10,Q33_Part_11,Q33_Part_12,Q33_OTHER_TEXT,Q34_Part_1,Q34_Part_2,Q34_Part_3,Q34_Part_4,Q34_Part_5,Q34_Part_6,Q34_Part_7,Q34_Part_8,Q34_Part_9,Q34_Part_10,Q34_Part_11,Q34_Part_12,Q34_OTHER_TEXT
0,Duration (in seconds),What is your age (# years)?,What is your gender? - Selected Choice,What is your gender? - Prefer to self-describe...,In which country do you currently reside?,What is the highest level of formal education ...,Select the title most similar to your current ...,Select the title most similar to your current ...,What is the size of the company where you are ...,Approximately how many individuals are respons...,Does your current employer incorporate machine...,Select any activities that make up an importan...,Select any activities that make up an importan...,Select any activities that make up an importan...,Select any activities that make up an importan...,Select any activities that make up an importan...,Select any activities that make up an importan...,Select any activities that make up an importan...,Select any activities that make up an importan...,Select any activities that make up an importan...,What is your current yearly compensation (appr...,Approximately how much money have you spent on...,Who/what are your favorite media sources that ...,Who/what are your favorite media sources that ...,Who/what are your favorite media sources that ...,Who/what are your favorite media sources that ...,Who/what are your favorite media sources that ...,Who/what are your favorite media sources that ...,Who/what are your favorite media sources that ...,Who/what are your favorite media sources that ...,Who/what are your favorite media sources that ...,Who/what are your favorite media sources that ...,Who/what are your favorite media sources that ...,Who/what are your favorite media sources that ...,Who/what are your favorite media sources that ...,On which platforms have you begun or completed...,On which platforms have you begun or completed...,On which platforms have you begun or completed...,On which platforms have you begun or completed...,On which platforms have you begun or completed...,...,Which specific big data / analytics products d...,Which of the following machine learning produc...,Which of the following machine learning produc...,Which of the following machine learning produc...,Which of the following machine learning produc...,Which of the following machine learning produc...,Which of the following machine learning produc...,Which of the following machine learning produc...,Which of the following machine learning produc...,Which of the following machine learning produc...,Which of the following machine learning produc...,Which of the following machine learning produc...,Which of the following machine learning produc...,Which of the following machine learning produc...,Which automated machine learning tools (or par...,Which automated machine learning tools (or par...,Which automated machine learning tools (or par...,Which automated machine learning tools (or par...,Which automated machine learning tools (or par...,Which automated machine learning tools (or par...,Which automated machine learning tools (or par...,Which automated machine learning tools (or par...,Which automated machine learning tools (or par...,Which automated machine learning tools (or par..

In [12]:
kaggle = kaggle_multiple_choice.iloc[1:,:]
kaggle.head()

,Time from Start to Finish (seconds),Q1,Q2,Q2_OTHER_TEXT,Q3,Q4,Q5,Q5_OTHER_TEXT,Q6,Q7,Q8,Q9_Part_1,Q9_Part_2,Q9_Part_3,Q9_Part_4,Q9_Part_5,Q9_Part_6,Q9_Part_7,Q9_Part_8,Q9_OTHER_TEXT,Q10,Q11,Q12_Part_1,Q12_Part_2,Q12_Part_3,Q12_Part_4,Q12_Part_5,Q12_Part_6,Q12_Part_7,Q12_Part_8,Q12_Part_9,Q12_Part_10,Q12_Part_11,Q12_Part_12,Q12_OTHER_TEXT,Q13_Part_1,Q13_Part_2,Q13_Part_3,Q13_Part_4,Q13_Part_5,...,Q31_OTHER_TEXT,Q32_Part_1,Q32_Part_2,Q32_Part_3,Q32_Part_4,Q32_Part_5,Q32_Part_6,Q32_Part_7,Q32_Part_8,Q32_Part_9,Q32_Part_10,Q32_Part_11,Q32_Part_12,Q32_OTHER_TEXT,Q33_Part_1,Q33_Part_2,Q33_Part_3,Q33_Part_4,Q33_Part_5,Q33_Part_6,Q33_Part_7,Q33_Part_8,Q33_Part_9,Q33_Part_10,Q33_Part_11,Q33_Part_12,Q33_OTHER_TEXT,Q34_Part_1,Q34_Part_2,Q34_Part_3,Q34_Part_4,Q34_Part_5,Q34_Part_6,Q34_Part_7,Q34_Part_8,Q34_Part_9,Q34_Part_10,Q34_Part_11,Q34_Part_12,Q34_OTHER_TEXT
1,510,22-24,Male,-1,France,Master’s degree,Software Engineer,-1,"1000-9,999 employees",0,I do not know,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,"30,000-39,999",$0 (USD),Twitter (data science influencers),NaN,NaN,"Kaggle (forums, blog, social media, etc)",NaN,NaN,NaN,"Blogs (Towards Data Science, Medium, Analytics...",Journal Publications (traditional publications...,NaN,NaN,NaN,-1,NaN,Coursera,NaN,DataCamp,NaN,...,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
2,423,40-44,Male,-1,India,Professional degree,Software Engineer,-1,"> 10,000 employees",20+,"We have well established ML methods (i.e., mod...",Analyze and understand data to influence produ...,Build and/or run the data infrastructure that ...,Build prototypes to explore applying machine l...,Build and/or run a machine learning service th...,NaN,NaN,NaN,NaN,-1,"5,000-7,499","> $100,000 ($USD)",NaN,NaN,NaN,"Kaggle (forums, blog, social media, etc)",NaN,"YouTube (Cloud AI Adventures, Siraj Raval, etc)","Podcasts (Chai Time Data Science, Linear Digre...","Blogs (Towards Data Science, Medium, Analytics...",NaN,NaN,NaN,NaN,-1,NaN,Coursera,NaN,DataCamp,NaN,...,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
3,83,55-59,Female,-1,Germany,Professional degree,NaN,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,NaN,NaN,NaN,NaN,NaN,...,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
4,391,40-44,Male,-1,Australia,Master’s degree,Other,0,"> 10,000 employees",20+,I do not know,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,"250,000-299,999","$10,000-$99,999",NaN,NaN,NaN,NaN,NaN,NaN,"Podcasts (Chai Time Data Science, Linear Digre...","Blogs (Towards Data Science, Medium, Analytics...",Journal Publications (traditional publications...,"Slack Communities (ods.ai, kagglenoobs, etc)",NaN,NaN,-1,NaN,Coursera,edX,DataCamp,NaN,...,-1,NaN,NaN,Azure Machine Learning Studio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Azure SQL Database,NaN,NaN,NaN,-1
5,392,22-24,Male,-1,India,Bachelor’s degree,Other,1,0-49 employees,0,No (we do not use ML methods),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,"4,000-4,999",$0 (USD),NaN,NaN,NaN,NaN,NaN,"YouTube (Cloud AI Adventures, Siraj Raval, etc)",NaN,"Blogs (Towards Data Science, Medium, Analytics...",NaN,NaN,NaN,Other,0,NaN,NaN,NaN,NaN,NaN,...,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1


Separate questions from answers

In [13]:
kaggle_questions = kaggle_multiple_choice.iloc[0,:]
kaggle_questions.head()

Time from Start to Finish (seconds)                                Duration (in seconds)
Q1                                                           What is your age (# years)?
Q2                                                What is your gender? - Selected Choice
Q2_OTHER_TEXT                          What is your gender? - Prefer to self-describe...
Q3                                             In which country do you currently reside?
Name: 0, dtype: object

#### Data cleaning

Remove surveys that take less than 3 minutes or more than 600 minutes (These surveys are not so valuable)

In [14]:
less_3_minutes = kaggle[round(kaggle.iloc[:,0].astype(int) / 60) <= 3].index
kaggle = kaggle.drop(less_3_minutes, axis=0)

more_600_minutes = kaggle[round(kaggle.iloc[:,0].astype(int) / 60) >= 600].index
kaggle = kaggle.drop(more_600_minutes, axis=0)

Remove respondents who are not employed or project/product managers

In [15]:
students_and_others = kaggle[(kaggle.Q5 == 'Student') | (kaggle.Q5 == 'Other') | (kaggle.Q5 == 'Not employed') | (kaggle.Q5 == 'Product/Project Manager')].index

kaggle = kaggle.drop(list(students_and_others), axis=0)

Remove those who didn't disclose compensation (Q10 is `NaN`)

In [16]:
kaggle.dropna(subset=['Q10'], inplace=True)

#### Grouping

In [17]:
kaggle.Q5.value_counts()

Data Scientist           3377
Software Engineer        2164
Data Analyst             1284
Research Scientist       1157
Business Analyst          602
Data Engineer             503
Statistician              236
DBA/Database Engineer     129
Name: Q5, dtype: int64

Group DBA and Data engineer together as one job title

In [18]:
kaggle.Q5 = kaggle.Q5.replace('DBA/Database Engineer', 'Data Engineer/DBA')
kaggle.Q5 = kaggle.Q5.replace('Data Engineer', 'Data Engineer/DBA')
kaggle.Q5.value_counts()

Data Scientist        3377
Software Engineer     2164
Data Analyst          1284
Research Scientist    1157
Data Engineer/DBA      632
Business Analyst       602
Statistician           236
Name: Q5, dtype: int64

Groupping statistician and Research Scientist:

In [19]:
kaggle.Q5 = kaggle.Q5.replace('Statistician', 'Statistician/Research Scientist')
kaggle.Q5 = kaggle.Q5.replace('Research Scientist', 'Statistician/Research Scientist')
kaggle.Q5.value_counts()

Data Scientist                     3377
Software Engineer                  2164
Statistician/Research Scientist    1393
Data Analyst                       1284
Data Engineer/DBA                   632
Business Analyst                    602
Name: Q5, dtype: int64

#### Process country names

In [20]:
kaggle.Q3 = kaggle.Q3.replace('United Kingdom of Great Britain and Northern Ireland', 'United Kingdom')
kaggle.Q3 = kaggle.Q3.replace('United States of America', 'United States')

#### Process programming languages (Q18)

In [21]:
kaggle['ProgLang'] = kaggle[['Q18_Part_1', 'Q18_Part_2', 'Q18_Part_3', 'Q18_Part_4', 'Q18_Part_5',
                             'Q18_Part_6', 'Q18_Part_7', 'Q18_Part_8', 'Q18_Part_9', 'Q18_Part_10', 
                             'Q18_Part_11', 'Q18_Part_12']].values.tolist()
kaggle.ProgLang.head()

1    [Python, R, SQL, nan, nan, Java, Javascript, n...
2    [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...
6    [Python, R, nan, nan, nan, nan, nan, nan, nan,...
7    [Python, R, nan, nan, nan, nan, nan, nan, Bash...
8    [Python, nan, nan, nan, nan, nan, nan, nan, na...
Name: ProgLang, dtype: object

In [22]:
# remove nulls
kaggle.ProgLang = kaggle.ProgLang.apply(lambda x: [item for item in x if not pd.isnull(item)])
kaggle.ProgLang.head()

1    [Python, R, SQL, Java, Javascript, MATLAB]
2                                            []
6                                   [Python, R]
7                             [Python, R, Bash]
8                                      [Python]
Name: ProgLang, dtype: object

Calculates the quantity of different Programming Languages

In [23]:
kaggle['QtyProgLang'] = kaggle.ProgLang.apply(lambda x: len(x))

# If Quantity > 6 then it will be 6
kaggle.loc[kaggle.QtyProgLang > 6, 'QtyProgLang'] = 6
kaggle.QtyProgLang.head()

1    6
2    0
6    2
7    3
8    1
Name: QtyProgLang, dtype: int64

#### Process cloud platforms (Q29)

In [24]:
kaggle['CloudPlatf']= kaggle[['Q29_Part_1', 'Q29_Part_2', 'Q29_Part_3', 'Q29_Part_4',
                            'Q29_Part_5', 'Q29_Part_6', 'Q29_Part_7', 'Q29_Part_8',
                            'Q29_Part_9', 'Q29_Part_10', 'Q29_Part_11', 'Q29_Part_12']].values.tolist()

# remove nulls
kaggle.CloudPlatf = kaggle.CloudPlatf.apply(lambda x: [item.strip().lower() for item in x if not pd.isnull(item)])

# Calculates the quantity
kaggle['QtyCloudPlatf'] = kaggle.CloudPlatf.apply(lambda x: len(x))
kaggle.loc[kaggle.QtyCloudPlatf > 6, 'QtyCloudPlatf'] = 6

#### Process databases (Q34)

In [25]:
kaggle['Databases']= kaggle[['Q34_Part_1', 'Q34_Part_2', 'Q34_Part_3', 'Q34_Part_4', 'Q34_Part_5', 'Q34_Part_6',
                             'Q34_Part_7', 'Q34_Part_8', 'Q34_Part_9', 'Q34_Part_10', 'Q34_Part_11', 'Q34_Part_12']].values.tolist()
kaggle.Databases = kaggle.Databases.apply(lambda x: [item.strip().lower() for item in x if not pd.isnull(item)])
kaggle['QtyDatabases'] = kaggle.Databases.apply(lambda x: len(x))
kaggle.loc[kaggle.QtyDatabases > 6, 'QtyDatabases'] = 6

#### Rename some columns

In [26]:
kaggle.columns = kaggle.columns.str.replace('Q15', 'TimeWritingCode')
kaggle.columns = kaggle.columns.str.replace('Q10', 'Salary')
kaggle.columns = kaggle.columns.str.replace('Q1', 'Age')
kaggle.columns = kaggle.columns.str.replace('Q5', 'JobTitle')
kaggle.columns = kaggle.columns.str.replace('Q3', 'Country')
kaggle.columns = kaggle.columns.str.replace('Q6', 'CompanySize')

#### Data transformation

Transform some columns into categories

In [27]:
# Transform TimeWritingCode column into category

time_writting_code = ['I have never written code', '< 1 years', '1-2 years', '3-5 years',
                      '5-10 years', '10-20 years', '20+ years']
cat_dtype = pd.api.types.CategoricalDtype(categories=time_writting_code, ordered=True)
kaggle.TimeWritingCode = kaggle.TimeWritingCode.astype(cat_dtype)

In [28]:
# Transform CompanySize into category

company_size = ['0-49 employees', '50-249 employees', '250-999 employees', '1000-9,999 employees', '> 10,000 employees']
cat_dtype = pd.api.types.CategoricalDtype(categories=company_size, ordered=True)
kaggle.CompanySize = kaggle.CompanySize.astype(cat_dtype)

In [29]:
# Transform JobTitle into category

job_titles = ['Business Analyst', 'Data Analyst', 'Data Scientist', 
              'Data Engineer/DBA', 'Software Engineer', 'Statistician/Research Scientist']
cat_dtype = pd.api.types.CategoricalDtype(categories=job_titles, ordered=True)
kaggle.JobTitle = kaggle.JobTitle.astype(cat_dtype)

Add count column to make groupby easier

In [30]:
kaggle['Count'] = 1

Transform range of salaries into numerical value

In [31]:
compensation = kaggle.Salary.str.replace(r'(?:(?!\d|\-).)*', '').str.replace('500000', '500-500000').str.split('-')
kaggle.Salary = compensation.apply(lambda x: (int(x[0]) * 1000 + int(x[1]))/ 2) / 1000 # calculated in thousand dollars

Transform range of ages into numerical value

In [32]:
age = kaggle.Age.str.replace(r'(?:(?!\d|\-).)*', '').str.replace('70', '70-80').str.split('-')
kaggle.Age = (age.apply(lambda x: (int(x[0]) + int(x[1]))/ 2)).astype(int)

#### Filter only the columns we're interested in

In [33]:
kaggle = kaggle[['Age', 'Country', 'JobTitle', 'CompanySize', 'Salary', 'TimeWritingCode', 'ProgLang',
                 'QtyProgLang', 'CloudPlatf', 'QtyCloudPlatf', 'Databases', 'QtyDatabases', 'Count']]

kaggle.head(10)

,Age,Country,JobTitle,CompanySize,Salary,TimeWritingCode,ProgLang,QtyProgLang,CloudPlatf,QtyCloudPlatf,Databases,QtyDatabases,Count
1,23,France,Software Engineer,"1000-9,999 employees",15019.9995,1-2 years,"[Python, R, SQL, Java, Javascript, MATLAB]",6,[],0,[],0,1
2,42,India,Software Engineer,"> 10,000 employees",2503.7495,I have never written code,[],0,[],0,[],0,1
6,52,France,Data Scientist,0-49 employees,30034.9995,20+ years,"[Python, R]",2,[amazon web services (aws)],1,"[postgressql, aws relational database service]",2,1
7,23,India,Data Scientist,50-249 employees,5007.4995,3-5 years,"[Python, R, Bash]",3,"[google cloud platform (gcp), amazon web servi...",3,"[mysql, postgressql]",2,1
8,23,United States,Data Scientist,"> 10,000 employees",40044.9995,3-5 years,[Python],1,[],0,[],0,1
12,32,Germany,Statistician/Research Scientist,0-49 employees,1001.4995,5-10 years,[R],1,[google cloud platform (gcp)],1,[mysql],1,1
13,32,Germany,Data Scientist,50-249 employees,35039.9995,5-10 years,"[Python, R]",2,[amazon web services (aws)],1,[postgressql],1,1
16,52,United States,Data Analyst,50-249 employees,62574.9995,10-20 years,"[Python, R, SQL]",3,[salesforce cloud],1,[microsoft sql server],1,1
17,19,Australia,Software Engineer,0-49 employees,0.4995,1-2 years,"[Python, R, SQL, C]",4,[amazon web services (aws)],1,[mysql],1,1
20,27,France,Statistician/Research Scientist,50-249 employees,20024.9995,5-10 years,"[Python, R]",2,[],0,[],0,1


In [34]:
kaggle.to_csv("data/multiple_choice_responses_preprocessed.csv")

In [35]:
pd.read_csv("https://gist.githubusercontent.com/EckoTan0804/7ba61515d185c6558f77504044b485bb/raw/4caac4c296138e0d40aa22c90ae38d712ba0531d/multiple_choice_responses_preprocessed.csv").head()

,Unnamed: 0,Age,Country,JobTitle,CompanySize,Salary,TimeWritingCode,ProgLang,QtyProgLang,CloudPlatf,QtyCloudPlatf,Databases,QtyDatabases,Count
0,1,23,France,Software Engineer,"1000-9,999 employees",15019.9995,1-2 years,"['Python', 'R', 'SQL', 'Java', 'Javascript', '...",6,[],0,[],0,1
1,2,42,India,Software Engineer,"> 10,000 employees",2503.7495,I have never written code,[],0,[],0,[],0,1
2,6,52,France,Data Scientist,0-49 employees,30034.9995,20+ years,"['Python', 'R']",2,['amazon web services (aws)'],1,"['postgressql', 'aws relational database servi...",2,1
3,7,23,India,Data Scientist,50-249 employees,5007.4995,3-5 years,"['Python', 'R', 'Bash']",3,"['google cloud platform (gcp)', 'amazon web se...",3,"['mysql', 'postgressql']",2,1
4,8,23,United States,Data Scientist,"> 10,000 employees",40044.9995,3-5 years,['Python'],1,[],0,[],0,1


### Glassdoor

In [36]:
glassdoor_full = pd.read_csv('data/glassdoor.csv')
glassdoor_full.head()

,benefits.benefitRatingDecimal,benefits.comments,benefits.highlights,benefits.numRatings,benefits.employerSummary,breadCrumbs,gaTrackerData.category,gaTrackerData.empId,gaTrackerData.empName,gaTrackerData.empSize,gaTrackerData.expired,gaTrackerData.industry,gaTrackerData.industryId,gaTrackerData.jobId.long,gaTrackerData.jobId.int,gaTrackerData.jobTitle,gaTrackerData.location,gaTrackerData.locationId,gaTrackerData.locationType,gaTrackerData.pageRequestGuid.guid,gaTrackerData.pageRequestGuid.guidValid,gaTrackerData.pageRequestGuid.part1,gaTrackerData.pageRequestGuid.part2,gaTrackerData.sector,gaTrackerData.sectorId,gaTrackerData.profileConversionTrackingParams.trackingCAT,gaTrackerData.profileConversionTrackingParams.trackingSRC,gaTrackerData.profileConversionTrackingParams.trackingXSP,gaTrackerData.jobViewTrackingResult.jobViewDisplayTimeMillis,gaTrackerData.jobViewTrackingResult.requiresTracking,gaTrackerData.jobViewTrackingResult.trackingUrl,header.adOrderId,header.advertiserType,header.applicationId,header.applyButtonDisabled,header.applyUrl,header.blur,header.coverPhoto,header.easyApply,header.employerId,...,salary.country.continent.id,salary.country.continent.new,salary.country.countryFIPS,salary.country.currency.currencyCode,salary.country.currency.defaultFractionDigits,salary.country.currency.displayName,salary.country.currency.id,salary.country.currency.name,salary.country.currency.negativeTemplate,salary.country.currency.new,salary.country.currency.positiveTemplate,salary.country.currency.symbol,salary.country.currencyCode,salary.country.defaultLocale,salary.country.defaultName,salary.country.defaultShortName,salary.country.employerSolutionsCountry,salary.country.id,salary.country.longName,salary.country.major,salary.country.name,salary.country.new,salary.country.population,salary.country.shortName,salary.country.tld,salary.country.type,salary.country.uniqueName,salary.country.usaCentricDisplayName,salary.currency.currencyCode,salary.currency.defaultFractionDigits,salary.currency.displayName,salary.currency.id,salary.currency.name,salary.currency.negativeTemplate,salary.currency.new,salary.currency.positiveTemplate,salary.currency.symbol,salary.lastSalaryDate,salary.salaries,wwfu
0,0.0,2801.0,2801.0,0,NaN,2801,-1,2183810,Modus Group,501-1000,False,Investment Banking & Asset Management,200053,3.227100e+09,NaN,Biogas Project Development Manager,Warsaw,3094484,CITY,0000016e56d0b5f2b16c3aea720c55ae,True,1573414548978,-5662085852901386834,Finance,10010,NaN,NaN,NaN,NaN,NaN,NaN,14295,GENERAL,0,False,/partner/jobListing.htm?pos=101&ao=14295&tgt=A...,True,/static/img/banners/default-ei-banner.jpg,True,2183810,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2483.0,NaN
1,4.0,2802.0,2802.0,3,NaN,2802,-1,10416,Boehringer Ingelheim,10000--1,False,Biotech & Pharmaceuticals,200021,3.406582e+09,NaN,Quality Manager - Boehringer Ingelheim Healthc...,NaN,3183562,CITY,0000016e56d0b5cfa19a4b2ba225a2ee,True,1573414548943,-6802041636413660434,Biotech & Pharmaceuticals,10005,NaN,NaN,NaN,NaN,NaN,NaN,43297,GENERAL,0,False,/partner/jobListing.htm?pos=101&ao=43297&s=58&...,True,/static/img/banners/default-ei-banner.jpg,False,10416,...,4.0,False,UK,GBP,2.0,United Kingdom Pound (GBP),2.0,United Kingdom Pound,$(#),False,$#,&pound;,GBP,en_UK,United Kingdom,United Kingdom,True,2.0,United Kingdom,True,United Kingdom,False,60363602.0,UK,.uk,N,United Kingdom,United Kingdom,GBP,2.0,United Kingdom Pound (GBP),2.0,United Kingdom Pound,$(#),False,$#,&pound;,2019-10-14T06:05:58,2484.0,NaN
2,0.0,2803.0,2803.0,0,NaN,2803,-1,1043373,Immobiliare.it,201-500,False,NaN,0,3.230738e+09,NaN,"Senior Software Engineer (PHP, Elixir, Python)",Milano,2802090,CITY,0000016e56d0b5c58f39733898e10986,True,1573414548933,-8126337365693101690,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,437149,GENERAL,0,False,/partner/jobListing.htm?pos=101&ao=437149&s=58...,True,/static/img/banners/default-

#### Select and rename columns

We select only the columns we're interested in: `header.jotTitle`, `job.description`, `map.country`

In [37]:
COLUMNS = ["header.jobTitle", "job.description", "map.country"]
glassdoor = glassdoor_full[COLUMNS].copy()

Rename selected columns to meaningful names:

In [38]:
glassdoor.columns = ["JobTitle", "JobDescription", "Country"]
glassdoor.head()

,JobTitle,JobDescription,Country
0,Biogas Project Development Manager,We are the drivers of the future who offer a g...,Poland
1,Quality Manager - Boehringer Ingelheim Healthc...,<strong></strong>\n<br/><br/>\n<strong></stron...,NaN
2,"Senior Software Engineer (PHP, Elixir, Python)",Dottori.it &egrave; il motore di ricerca che t...,Italy
3,Senior SQA Engineer,Codup is looking for a Senior SQA Engineer for...,PK
4,Research Manager,<strong>Department:</strong> Research &amp; Op...,IN


#### Processing `Country` column

Drop `NaN` countries:

In [39]:
glassdoor.dropna(subset=["Country"], inplace=True)

The value in `Country` column is not consistent: some country names are written in full, while others are in 2 digit codes.

In [40]:
glassdoor["Country"]

0          Poland
2           Italy
3              PK
4              IN
6              IL
           ...   
165284    Hungary
165286         JP
165287         BE
165288         NL
165289         SG
Name: Country, Length: 115262, dtype: object

We fix this problem using `country_names_2_digit_codes.csv`

In [41]:
country_codes = pd.read_csv("data/country_names_2_digit_codes.csv")
country_codes.head()

,Name,Code
0,Afghanistan,AF
1,Ã…land Islands,AX
2,Albania,AL
3,Algeria,DZ
4,American Samoa,AS


Merge both datasets by 2 digit code, and fill `NaN`s with full country name:

In [42]:
glassdoor = pd.merge(glassdoor, country_codes, left_on='Country', right_on='Code', how='left')
glassdoor.head()

,JobTitle,JobDescription,Country,Name,Code
0,Biogas Project Development Manager,We are the drivers of the future who offer a g...,Poland,NaN,NaN
1,"Senior Software Engineer (PHP, Elixir, Python)",Dottori.it &egrave; il motore di ricerca che t...,Italy,NaN,NaN
2,Senior SQA Engineer,Codup is looking for a Senior SQA Engineer for...,PK,Pakistan,PK
3,Research Manager,<strong>Department:</strong> Research &amp; Op...,IN,India,IN
4,B2B Product Manager,<b>&#1514;&#1497;&#1488;&#1493;&#1512; &#1492;...,IL,Israel,IL


Then replace the 2 digit codes with full name:

In [43]:
glassdoor["Country"] = glassdoor["Name"].fillna(glassdoor["Country"])
glassdoor.head()

,JobTitle,JobDescription,Country,Name,Code
0,Biogas Project Development Manager,We are the drivers of the future who offer a g...,Poland,NaN,NaN
1,"Senior Software Engineer (PHP, Elixir, Python)",Dottori.it &egrave; il motore di ricerca che t...,Italy,NaN,NaN
2,Senior SQA Engineer,Codup is looking for a Senior SQA Engineer for...,Pakistan,Pakistan,PK
3,Research Manager,<strong>Department:</strong> Research &amp; Op...,India,India,IN
4,B2B Product Manager,<b>&#1514;&#1497;&#1488;&#1493;&#1512; &#1492;...,Israel,Israel,IL


Lastly, drop `Name` and `Code` columns:

In [44]:
glassdoor = glassdoor.drop(["Name", "Code"], axis=1)
glassdoor.head()

,JobTitle,JobDescription,Country
0,Biogas Project Development Manager,We are the drivers of the future who offer a g...,Poland
1,"Senior Software Engineer (PHP, Elixir, Python)",Dottori.it &egrave; il motore di ricerca che t...,Italy
2,Senior SQA Engineer,Codup is looking for a Senior SQA Engineer for...,Pakistan
3,Research Manager,<strong>Department:</strong> Research &amp; Op...,India
4,B2B Product Manager,<b>&#1514;&#1497;&#1488;&#1493;&#1512; &#1492;...,Israel


Remove some countries that didn't match official nomenclature

In [45]:
glassdoor = pd.merge(glassdoor, country_codes, left_on='Country', right_on='Name', how='left')
glassdoor.head()

,JobTitle,JobDescription,Country,Name,Code
0,Biogas Project Development Manager,We are the drivers of the future who offer a g...,Poland,Poland,PL
1,"Senior Software Engineer (PHP, Elixir, Python)",Dottori.it &egrave; il motore di ricerca che t...,Italy,Italy,IT
2,Senior SQA Engineer,Codup is looking for a Senior SQA Engineer for...,Pakistan,Pakistan,PK
3,Research Manager,<strong>Department:</strong> Research &amp; Op...,India,India,IN
4,B2B Product Manager,<b>&#1514;&#1497;&#1488;&#1493;&#1512; &#1492;...,Israel,Israel,IL


In [46]:
glassdoor.dropna(subset=['Name'], inplace=True)
glassdoor = glassdoor.drop(['Name', 'Code'], axis=1)

#### Processing `JobTitle`


Job titles we're interested in:

In [47]:
JOB_TITLES = [
    'data scientist', 
    'software engineer', 
    'data analyst', 
    'research scientist', 
    'business analyst',
    'data engineer', 
    'statistician', 
    'dba', 
    'database engineer', 
    'machine learning engineer'
]

In [48]:
import re
import numpy as np

job_masks = [glassdoor.JobTitle.str.contains(job_title, flags=re.IGNORECASE, regex=True) for job_title in JOB_TITLES]

combined_mask = np.vstack(job_masks).any(axis=0)
glassdoor = glassdoor[combined_mask].reset_index(drop=True)
glassdoor.head()

,JobTitle,JobDescription,Country
0,"Senior Software Engineer (PHP, Elixir, Python)",Dottori.it &egrave; il motore di ricerca che t...,Italy
1,Big Data Analyst,Amaris est une soci&eacute;t&eacute; ind&eacut...,Belgium
2,Business Analyst IFRS 17 Implementation Paris F/H,<div>Business Analyst Paris F/H</div>\n<br/>\n...,France
3,Data Scientist media,<ul>\n<li>*Data Scientist media**</li>\n<li>*L...,United Kingdom
4,Data Scientist,<strong>Job Title:</strong> Data Scientist\n<b...,Spain


In [49]:
job_titles_regex = '|'.join(JOB_TITLES)
glassdoor.JobTitle = glassdoor.JobTitle.str.findall(job_titles_regex, flags=re.IGNORECASE)
glassdoor.JobTitle = glassdoor.JobTitle.str[0]
glassdoor.JobTitle = glassdoor.JobTitle.str.title()
glassdoor.head()

,JobTitle,JobDescription,Country
0,Software Engineer,Dottori.it &egrave; il motore di ricerca che t...,Italy
1,Data Analyst,Amaris est une soci&eacute;t&eacute; ind&eacut...,Belgium
2,Business Analyst,<div>Business Analyst Paris F/H</div>\n<br/>\n...,France
3,Data Scientist,<ul>\n<li>*Data Scientist media**</li>\n<li>*L...,United Kingdom
4,Data Scientist,<strong>Job Title:</strong> Data Scientist\n<b...,Spain


Rename some job titles:

In [50]:
glassdoor.JobTitle = glassdoor.JobTitle.replace('Dba', 'Data Engineer/DBA')
glassdoor.JobTitle = glassdoor.JobTitle.replace('Database Engineer', 'Data Engineer/DBA')
glassdoor.JobTitle = glassdoor.JobTitle.replace('Data Engineer', 'Data Engineer/DBA')

# group Statistician + Research Scientist
glassdoor.JobTitle = glassdoor.JobTitle.replace('Statistician', 'Statistician/Research Scientist')
glassdoor.JobTitle = glassdoor.JobTitle.replace('Research Scientist', 'Statistician/Research Scientist')

glassdoor.JobTitle.value_counts()

Software Engineer                  10825
Business Analyst                    6719
Data Scientist                      4561
Data Engineer/DBA                   4178
Data Analyst                        4171
Statistician/Research Scientist      710
Machine Learning Engineer            499
Name: JobTitle, dtype: int64

Then we tranform `JobTitle` column into category:

In [51]:
job_titles = ['Business Analyst', 'Data Analyst', 'Data Scientist', 
              'Data Engineer/DBA', 'Software Engineer', 'Statistician/Research Scientist']
cat_dtype = pd.api.types.CategoricalDtype(categories=job_titles, ordered=True)
glassdoor.JobTitle = glassdoor.JobTitle.astype(cat_dtype)

In [52]:
# Add column to make groupby easier
glassdoor['Count'] = 1

In [53]:
glassdoor.head()

,JobTitle,JobDescription,Country,Count
0,Software Engineer,Dottori.it &egrave; il motore di ricerca che t...,Italy,1
1,Data Analyst,Amaris est une soci&eacute;t&eacute; ind&eacut...,Belgium,1
2,Business Analyst,<div>Business Analyst Paris F/H</div>\n<br/>\n...,France,1
3,Data Scientist,<ul>\n<li>*Data Scientist media**</li>\n<li>*L...,United Kingdom,1
4,Data Scientist,<strong>Job Title:</strong> Data Scientist\n<b...,Spain,1


#### Processing `JobDescription`

Make `JobDescription` lower case:

In [54]:
glassdoor.JobDescription = glassdoor.JobDescription.str.lower()

Find metions to cloud platforms and databases in `JobDescription`:

In [55]:
cloud_platforms = {
    'Alibaba': ' Alibaba Cloud ', 
    'Amazon Web Services': ' Amazon Web Services (AWS) ',
    'AWS': ' Amazon Web Services (AWS) ',
    'Google Cloud Platform': ' Google Cloud Platform (GCP) ', 
    'GCP': ' Google Cloud Platform (GCP) ',
    'Google Cloud': ' Google Cloud Platform (GCP) ',
    'IBM': ' IBM Cloud ', 
    'Azure': ' Microsoft Azure ', 
    'Oracle': ' Oracle Cloud ',
    'Red Hat': ' Red Hat Cloud ',
    'SAP': ' SAP Cloud ', 
    'Salesforce': ' Salesforce Cloud ', 
    'VMware': ' VMware Cloud '
}

# Replacing terms into Job Description
for find, repl in cloud_platforms.items():
    glassdoor.JobDescription = glassdoor.JobDescription.str.replace(find.lower(), repl.lower())

In [56]:
databases ={
    'dynamodb': ' aws dynamodb ',
    'dynamo': ' aws dynamodb ',
    ' rds ': ' aws relational database service ',
    'relational database service': ' aws relational database service ',
    'azure sql': ' azure sql database ',
    'google cloud sql': ' google cloud sql ',
    'microsoft access': ' microsoft access ', 
    'sql server': ' microsoft sql server ', 
    'my sql': ' mysql ', 
    'oracle db': ' oracle database ', 
    'postgres': ' postgressql ',
    'postgre': ' postgressql ',
    'postgre sql': ' postgressql ',
    'sqlite': 'sqlite '
}

for find, repl in databases.items():
    glassdoor.JobDescription = glassdoor.JobDescription.str.replace(find.lower(), repl.lower())

In [57]:
glassdoor.to_csv("data/glassdoor_preprocessed.csv")

## Data visualization

### Proportion for each job title

In [58]:
glassdoor.head()

,JobTitle,JobDescription,Country,Count
0,Software Engineer,dottori.it &egrave; il motore di ricerca che t...,Italy,1
1,Data Analyst,amaris est une soci&eacute;t&eacute; ind&eacut...,Belgium,1
2,Business Analyst,<div>business analyst paris f/h</div>\n<br/>\n...,France,1
3,Data Scientist,<ul>\n<li>*data scientist media**</li>\n<li>*l...,United Kingdom,1
4,Data Scientist,<strong>job title:</strong> data scientist\n<b...,Spain,1


In [59]:
glassdoor_de = glassdoor[glassdoor.Country == "Germany"].groupby(["JobTitle"], as_index=False).Count.sum().Count.tolist()
glassdoor_de = (np.array(glassdoor_de) / sum(glassdoor_de) * 100).tolist()
glassdoor_de

[13.360739979445016,
 14.953751284686536,
 26.258992805755394,
 21.634121274409043,
 21.017471736896198,
 2.774922918807811]

In [60]:
glassdoor_fr = glassdoor[glassdoor.Country == "France"].groupby(["JobTitle"], as_index=False).Count.sum().Count.tolist()
glassdoor_fr = (np.array(glassdoor_fr) / sum(glassdoor_fr) * 100).tolist()
glassdoor_fr

[24.251012145748987,
 19.51417004048583,
 22.914979757085018,
 22.30769230769231,
 9.635627530364372,
 1.376518218623482]

In [61]:
df_de = pd.DataFrame(dict(r=glassdoor_de, theta=job_titles))
df_fr = pd.DataFrame(dict(r=glassdoor_fr, theta=job_titles))

In [62]:
import plotly.graph_objects as go

class PolarPlot():

    def __init__(self):
        self.figure = go.Figure() # instatiates plotly figure
        self.range = (0, 0) # define the initial range of polar plots
        self.theta = ['Business Analyst', 'Data Analyst', 'Data Scientist', 'Data Engineer/DBA',
                      'Software Engineer', 'Statistician/Research Scientist', 'Business Analyst'] # Those are the Theta values for our plot
    
    def update_common_layout(self):
        """
        Updates general layout characteristics
        """
        self.figure.update_layout(
            showlegend = True,
            legend_itemclick = 'toggleothers', 
            legend_itemdoubleclick = 'toggle', 
            width = 800,  
            height = 500 
        )
        
    def update_commom_polar_layout(self):
        """
        Updates polar layout characteristics
        """
        self.figure.update_layout(
            # polar_bgcolor='white', # White background is always better
            # polar_radialaxis_visible=True, # we want to show the axis
            # polar_radialaxis_showticklabels=True, # we want to show the axis titles
            # polar_radialaxis_tickfont_color='darkgrey', # grey to the axis label (Software Engineer, Data Scientist, etc)
            # polar_angularaxis_color='grey', # Always grey for all elements that are not important
            # polar_angularaxis_showline=False, # hide lines that are not necessary
            # polar_radialaxis_showline=False, # hide lines that are not necessary
            # polar_radialaxis_layer='below traces', # show the axis bellow all traces
            # polar_radialaxis_gridcolor='#F2F2F2', # grey to not draw attention
            polar_radialaxis_range=self.range # gets the range attribute, that is calculated in another method
        )
        
    def add_data(self, data, country, hover_template='%{r:0.0f}%'):
        """
        Adds a trace to the figure following the same standard for each trace
        """
        data.append(data[0]) # add the first element to the end of the list, this will "close" the polar chart 
        self.figure.add_trace(
            go.Scatterpolar(
                r=data, 
                theta=self.theta, 
                mode='lines', 
                name=country, 
                hoverinfo='name+r', 
                hovertemplate=hover_template, 
                showlegend=True, 
                line_shape='spline', 
                line_smoothing=0.8, 
                # line_width=1.6 
            )
        )
        self.update_range(data) # Calls the method that will update the max range
    
    def update_range(self, data):
        """
        Updates the range to be 110% of maximum value of all traces
        """
        max_range = max(data) * 1.1
        self.range = (0, max_range) if max_range > self.range[1] else self.range # updates the range attribute
        
    def show(self):
        """
        Update layouts and shows the figure
        """
        self.update_common_layout() 
        self.update_commom_polar_layout()
        self.figure.show()

In [63]:
pp = PolarPlot()

# example plot
pp.add_data(glassdoor_de, "Germany")
pp.add_data(glassdoor_fr, "France")
pp.show()

In [64]:
def remove_trace(fig, name):
    fig.data = tuple(trace for trace in fig.data if trace["name"] != name)

In [65]:
remove_trace(pp.figure, "France")
pp.show()

### Salaries

In [66]:
class LinePlot():

    def __init__(self):
        self.figure = go.Figure()
        self.range = (0, 100)
    
    def update_axis_title(self, x, y):
        self.figure.update_layout(
            xaxis_title_text=x,
            yaxis_title_text=y,
        )
        
    def update_layout(self):
        """
        Creates a clean layout for ploting, adjusting multiple settings
        """
        self.figure.update_layout(
            showlegend=True,
            legend_font_color='gray',
            legend_itemclick='toggleothers',
            legend_itemdoubleclick='toggle',
            width = 800,
            height=500,
            # plot_bgcolor='white',
            xaxis_title_font_color='grey',
            xaxis_color='grey',
            yaxis_title_font_color='grey',
            yaxis_color='grey',
        )
         
    def add_data(self, x_names, y_data, trace_name, hover_template):
        """
        Adds a trace to the figure following the same standard for each trace
        """
        self.figure.add_trace(
            go.Scatter(
                x=x_names,
                y=y_data,
                mode='lines',
                name=trace_name,
                hoverinfo='name+y',
                hovertemplate=hover_template,
                line_shape='spline',
                line_smoothing=0.8,
                line_width=1.6
            )
        )
        
    def show(self):
        self.update_layout()
        self.figure.show()

In [67]:
def plot_lines(line_plot, data, traces, x_names, agg_column, group_column, trace_column, hover_template):
    """
    Creates aggregation to plot
    """
    for trace_name in traces:
        data_filtered = data[data[trace_column] == trace_name] 
        plot_data = data_filtered.groupby([group_column], as_index=False).agg({agg_column: ['mean', 'count']}) 
        plot_data = plot_data[agg_column]['mean'].tolist() 
        line_plot.add_data(x_names, plot_data, trace_name, hover_template=hover_template) 

#### Time of coding

In [73]:
JOB_TITLES = ['Business Analyst', 'Data Analyst', 'Data Scientist', 'Data Engineer/DBA',
          'Software Engineer', 'Statistician/Research Scientist']

x_names = ['0 years', '< 1 years', '1-2 years', '3-5 years', '5-10 years', '10-20 years', '20+ years'] 

time_of_coding_line_plot = LinePlot()

plot_lines(
    time_of_coding_line_plot, 
    data=kaggle, 
    traces=JOB_TITLES, 
    x_names=x_names, 
    agg_column='Salary', 
    group_column='TimeWritingCode', 
    trace_column='JobTitle', 
    hover_template='U$%{y:,.2r}' 
)

xaxis_title='Time of writing code'
yaxis_title='Average Salary (USD per Year)'

time_of_coding_line_plot.update_axis_title(xaxis_title, yaxis_title) 
time_of_coding_line_plot.show()

#### Company size

In [75]:
salary_line_plot = LinePlot()

COMPANY_SIZE = [
    '0-49 employees', 
    '50-249 employees', 
    '250-999 employees', 
    '1000-9,999 employees', 
    '> 10,000 employees'
]

plot_lines(
    salary_line_plot, 
    data=kaggle,
    traces=JOB_TITLES, 
    x_names=COMPANY_SIZE, 
    agg_column='Salary', 
    group_column='CompanySize',
    trace_column='JobTitle', 
    hover_template='U$%{y:,.2r}'
)

xaxis_title='Company size'
yaxis_title='Average Salary (USD per Year)'
salary_line_plot.update_axis_title(xaxis_title, yaxis_title)
salary_line_plot.show()

### Job skills

In [76]:
def plot_polar(polar_plot, data, traces, x_names, agg_column, group_column, trace_column, hover_template):

    data_cp = data.copy() 

    for trace_name in traces:

        if agg_column in ('JobDescription', 'CloudPlatf'): 
            data_cp['TempCol'] = data_cp[agg_column].apply(lambda x: trace_name.lower() in x) 
        else: 
            data_cp['TempCol'] = data_cp[agg_column].apply(lambda x: trace_name in x) 

        plot_data = data_cp.groupby([group_column], as_index=False).agg({'TempCol': ['sum', 'count']}) 
        plot_data['TempColPct'] = plot_data['TempCol']['sum'] / plot_data['TempCol']['count'] * 100 
        plot_data = plot_data.TempColPct.tolist() 
        polar_plot.add_data(plot_data, trace_name, hover_template) 

#### Popularity of different programming languages in Kaggle

In [77]:
job_skills_polar_plot = PolarPlot()

SKILLS = ['Bash', 'C', 'C++', 'Java', 'Javascript', 'MATLAB', 
          'Other', 'Python', 'R', 'SQL', 'TypeScript'] 

plot_polar(
    job_skills_polar_plot, 
    data=kaggle,
    traces=SKILLS, 
    x_names=JOB_TITLES, 
    agg_column='ProgLang', 
    group_column='JobTitle',
    trace_column='ProgLang', 
    hover_template='%{r:0.0f}%'
)

job_skills_polar_plot.figure.update_layout(
    polar_radialaxis_tickvals=[25, 50, 75],
    polar_radialaxis_ticktext=['25%', '50%', '75%'],
    polar_radialaxis_tickmode='array',
)

job_skills_polar_plot.show()

#### Popularity of different programming languages in Job Description

In [78]:
job_desc_polar_plot = PolarPlot()

plot_polar(
    job_desc_polar_plot, 
    data=glassdoor,
    traces=SKILLS, 
    x_names=JOB_TITLES, 
    agg_column='JobDescription', 
    group_column='JobTitle',
    trace_column='JobDescription', 
    hover_template='%{r:0.0f}%'
)

job_desc_polar_plot.figure.update_layout(
    polar_radialaxis_tickvals=[25, 50, 75],
    polar_radialaxis_ticktext=['25%', '50%', '75%'],
    polar_radialaxis_tickmode='array',
)
job_desc_polar_plot.show()

### How many programming languages should you learn?

In [79]:
prog_language_line_plot = LinePlot()

traces = list(set(kaggle.TimeWritingCode.tolist()))
# traces = []

x_names = ['{} languages'.format(x) for x in range(7)]

plot_lines(
    prog_language_line_plot, 
    data=kaggle,
    traces=traces, 
    x_names=x_names, 
    agg_column='Salary', 
    group_column='QtyProgLang',
    trace_column='TimeWritingCode', 
    hover_template='U$%{y:,.2r}'
)

# Adding Averarage
plot_data = kaggle.groupby(['QtyProgLang'], as_index=False).agg({'Salary': 'mean'})
plot_data = plot_data.Salary.tolist()
prog_language_line_plot.add_data(x_names, plot_data, 'Average', hover_template='U$%{y:,.2r}')


xaxis_title='Quantity of programming languages'
yaxis_title='Average Salary (USD per Year)'
prog_language_line_plot.update_axis_title(xaxis_title, yaxis_title)
prog_language_line_plot.show()